![texto del vínculo](https://drive.google.com/uc?id=1TBzdvw2MsV52hOViza89_Zr-1LINY2Ur)

PROYECTO : CLASIFICACION USANDO NAIVE BAYES

INTEGRANTES DEL GRUPO:
1. XXXXXX , CARNET No.
2.
3. **texto en negrita**


1. IMPORTAR LIBRERIAS

In [1]:
# 1.1 ) Cargar la herramiento joblib que servira para guardar y descargar el modelo entrenado en Google Drive de tal manera que tengamos el modelo
 #disponible sin necesidad de correr todo el programa de la nueve del Colaboratory
!pip install joblib
 # Cargar la API de Kaggle para leer el dataset desde la web
!pip install kaggle

In [2]:
# 1.2 ) Importar las librerías estándar de análisis
# Libreria pandas servira crear los data frame asi como manipular y analizar datos
import pandas as pd
# Libreria numpy servira para manejar vectores y matrices de ser necesarios asi como matematica de alto nivel
import numpy as np
#from collections import defaultdict, esta libreria nos permitira crear diccionarios predetermiandos
from collections import defaultdict
# Permite cargar el  modelo entrenado de machine learning Naive Bayes a Google Drive
from joblib import dump


2. IMPORTAR DATOS

In [ ]:
# 2.1) Se define la ruta para leer los archivos a usar desde la nube Google Colaboratory
     # a Google Drive
BASE_FOLDER = '/content/drive/My Drive/Colab Notebooks/PROYECTO IA/'

# Instruccion que permite importar y montar una unidad de Drive de Google drive en Google colaboratory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 2.2) Ruta para leer el archivo en Google Drive
file_path = BASE_FOLDER+ "naivebayes.py"

# Verificamos si existe el archivo en Google Drive
import os
if os.path.exists(file_path):
    print("✅ Archivo encontrado en Google Drive")
else:
    print("❌ Archivo NO encontrado. Revisar la ruta.")

✅ Archivo encontrado en Google Drive


In [5]:
# 2.3) Importar la clase Naive Bayes desde Google Driver
import sys
sys.path.append(BASE_FOLDER)  # Ruta donde se encuentra la clase RecNBuilder y la funcion stratified_kfold

from naivebayes import RecNBBuilder, stratified_kfold

In [ ]:
# 2.4 ) Se ubica el archivo token generado por la cuenta Kaggle en Google drive
!mkdir -p ~/.kaggle
!cp "/content/drive/My Drive/Colab Notebooks/PROYECTO IA/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d kazanova/sentiment140
!unzip sentiment140.zip

# 2.5) La funcion importa los datos del archivo en CSV y lo guarda en unh DataFrame Pandas
df = pd.read_csv("training.1600000.processed.noemoticon.csv",
                 encoding="latin-1",
                 header=None)

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


# 3. EXPLORACION DE DATOS

In [7]:
# 3.1) # Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
# 3.2 ) # Con este metodo se explora la informacion del DataFrame de pandas
        # como por ejemplo tipo de indice ,numero de filas y columnas, tipo de variables, el numero de valores No nulos ,uso de
        # memoria etc.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   0       1600000 non-null  int64 
 1   1       1600000 non-null  int64 
 2   2       1600000 non-null  object
 3   3       1600000 non-null  object
 4   4       1600000 non-null  object
 5   5       1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


# 3.3) Informacion relevante obtenida de la exploracion del DataFrame
# Sentiment140:
#   0 => negative, 2 => neutral, 4 => positive
# columns: [target, id, date, flag, user, text]

In [9]:
# 3.3) Este metodo sirve para definir el nombre del encabezado de los atributos del DataFrame
df.columns = ["target","id","date","flag","user","text"]

# 3.4)  Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
# 3.5 ) Selecciona columnas relevantes 'target' y 'text'
df = df[["target","text"]]

# 3.6)  Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [11]:
# 3.7) Funcion para definir el mapa de los sentimientos en valores numericos enteros {0->neg, 2->neu, 4->pos}
def map_sentiment(value):
    if value == 0:
        return "neg"
    elif value == 2:
        return "neu"
    elif value == 4:
        return "pos"

 # 3.8) Se crea el Dataframe con los sentimientos categorizados por numeros enteros

df["sentiment"] = df["target"].apply(map_sentiment)

# 3.8.1 ) Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df.head()

,target,text,sentiment
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",neg
1,0,is upset that he can't update his Facebook by ...,neg
2,0,@Kenichan I dived many times for the ball. Man...,neg
3,0,my whole body feels itchy and like its on fire,neg
4,0,"@nationwideclass no, it's not behaving at all....",neg


In [12]:
# 3.9 ) Procedimiento para contar la frecuencia de cada valor unico en la columna  "sentiment"
df['sentiment'].value_counts()

,count
sentiment,
neg,800000
pos,800000


4. LIMPIEZA DE DATOS

In [13]:
# 4.1 ) Limpieza / Preprocesamiento
#    Funcion para limpiar texto de palabras raras: quitar URL, stopwords, etc.
import re
def basic_clean(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+','', text) # quita URLs
    text = re.sub(r'@\w+','', text)          # quita menciones
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text) # deja solo letras
    text = re.sub(r'\s+',' ', text).strip() #limina los espacios en blanco y caracteres espaciales
    #que hay tanto a la decrecha como a la izquierda del String "s".
    #Existen también las variantes de:
    #s.rstrip(): Elimina los espacios en blanco y caracteres espaciales que hay a la derecha del string.
    #s.lstrip(): Elimina los espacios en blanco y caracteres espaciales que hay a la izquierda del string.
    return text

# 4.2 ) Se crea un DataFrame limpio de caracteres raros
df["clean_text"] = df["text"].apply(basic_clean)

# 4.2.1) Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df["clean_text"].head()

,clean_text
0,a thats a bummer you shoulda got david carr of...
1,is upset that he cant update his facebook by t...
2,i dived many times for the ball managed to sav...
3,my whole body feels itchy and like its on fire
4,no its not behaving at all im mad why am i her...


5. PROCESAMIENTO DE DATOS

In [14]:
# 5.1 ) Funcion para Tokenización consiste en dividir el texto en partes mas pequeñas para facilitar el analisis
def Tokens(text):

    text = re.split(r'[ \t\n]+', text) # Tokeniza genera una lista
    #El uso de la r convierte a los caracteres especiales en parte del
    #texto y no en comandos ejecutables dentro de la cadena, lo que es esencial
    #cuando tratamos con texto que incluye caracteres de escape.
    #el uso de \s+ permite dividir el texto en base a uno o más espacios vacíos,
    #creando tokens individuales para cada palabra
    return text

# 5.2 ) Se crea un DataFrame de Tokens
df["Tokens"] = df["clean_text"].apply(Tokens)

# 5.2.1 ) Con este metodo de Data Frame Pandas se exploran los datos de las primeras cinco primer filas del
       #Data Set a partir de la posicion 0.
df["Tokens"].head()


,Tokens
0,"[a, thats, a, bummer, you, shoulda, got, david..."
1,"[is, upset, that, he, cant, update, his, faceb..."
2,"[i, dived, many, times, for, the, ball, manage..."
3,"[my, whole, body, feels, itchy, and, like, its..."
4,"[no, its, not, behaving, at, all, im, mad, why..."


6. ENTRENAMIENTO Y EVALUACION DEL ALGORITMO DE APRENDIZAJE

In [15]:
#  Convertir columnas del DataFrame a arrays numpy para procesamiento eficiente
tokens_array = np.array(df["Tokens"], dtype=object)  # Array de tokens/textos (features)
labels_array = np.array(df["sentiment"], dtype=object)  # Array de etiquetas/sentimientos (target)

#  Crear array con índices numéricos de todas las muestras (0 a N-1)
all_idx = np.arange(len(df))  # Ejemplo: [0, 1, 2, ..., 999] para 1000 muestras

#  Generar folds estratificados usando nuestra función personalizada

folds = stratified_kfold(y=df["sentiment"], k=5)  # 5 folds manteniendo proporción de clases

# 6.1) Evaluacion y Validacion del modelo de aprendizaje

# Lista para almacenar las precisiones de cada fold
accuracies = []


k=1 # Contador de iteraciones por cada Fold
# Iterar sobre cada fold (i = número de fold, test_idx = índices de test)
for i, test_idx in enumerate(folds, start=1):
    # Asegurar que los índices de test sean enteros para indexación válida
    test_idx = np.array(test_idx, dtype=int)

    # Obtener índices de entrenamiento como complemento de los de test
    train_idx = np.setdiff1d(all_idx, test_idx, assume_unique=True)  # Resta de conjuntos

    # Dividir datos usando los índices
    X_train = tokens_array[train_idx]  # Features de entrenamiento
    y_train = labels_array[train_idx]  # Labels de entrenamiento
    X_test  = tokens_array[test_idx]   # Features de test
    y_test  = labels_array[test_idx]   # Labels de test (ground truth)

# 6.2) Configurar y construir el modelo (Naive Bayes con suavizado de Laplace)
    builder = RecNBBuilder().set_laplace(1.0)  # Suavizado para evitar prob=0
    model   = builder.build()  # Instanciar modelo

# 6.3) Entrenar con los datos de entrenamiento del fold actual
    model.fit(X_train, y_train)  # Aprender patrones de los datos

# 6.4) Guardar el modelo en Google Drive sin comprimirlo ya que el archivo es menor a 100 MB
    dump(model, '/content/drive/My Drive/PROYECTO IA JR/modelo_entrenado.joblib')

# 6.5) Calcular precisión en el conjunto de test del fold
    report = model.metrics_report(X_test, y_test)


# 6.6) Imprimir métricas por clase
    print("Iteracion", k, ":")
    for clase, valores in report['per_class'].items():
       print(f"Clase {clase}:")
       print(f"  Precisión: {valores['precision']:.3f}")
       print(f"  Recall: {valores['recall']:.3f}")
       print(f"  F1: {valores['f1']:.3f}")

    # Contador + 1 de iteraciones por cada Fold
    k=k+1

# 6.7) Imprimir macro promedios
print("\nMacro promedios:")
print(f"Precisión: {report['macro']['precision']:.3f}")
print(f"Recall: {report['macro']['recall']:.3f}")
print(f"F1: {report['macro']['f1']:.3f}")

Iteracion 1 :
Clase pos:
  Precisión: 0.793
  Recall: 0.764
  F1: 0.778
Clase neg:
  Precisión: 0.772
  Recall: 0.800
  F1: 0.786
Iteracion 2 :
Clase pos:
  Precisión: 0.792
  Recall: 0.762
  F1: 0.777
Clase neg:
  Precisión: 0.771
  Recall: 0.800
  F1: 0.785
Iteracion 3 :
Clase pos:
  Precisión: 0.794
  Recall: 0.762
  F1: 0.777
Clase neg:
  Precisión: 0.771
  Recall: 0.802
  F1: 0.786
Iteracion 4 :
Clase pos:
  Precisión: 0.793
  Recall: 0.765
  F1: 0.779
Clase neg:
  Precisión: 0.773
  Recall: 0.801
  F1: 0.787
Iteracion 5 :
Clase pos:
  Precisión: 0.793
  Recall: 0.764
  F1: 0.778
Clase neg:
  Precisión: 0.772
  Recall: 0.801
  F1: 0.786

Macro promedios:
Precisión: 0.783
Recall: 0.782
F1: 0.782


7. PREDICCION DE LA CLASE

In [ ]:
# 7.1 )
# Permite cargar el  modelo entrenado de machine learning Naive Bayes en Google Drive
from joblib import load
# Asegurar que la clase esté disponible
from naivebayes import RecNaiveBayes  # Asegurar que la clase esté disponible

#7.1.1 ) # Cargar el modelo guardado en Google Drive
modelo_cargado = load('/content/drive/My Drive/Colab Notebooks/PROYECTO IA/modelo_entrenado.joblib')

#7.2 Comprobacion con tweet fuera de prueba
tweet_nuevo = ["I'm at the office working on a new project."]
prediccion = modelo_cargado.predict(tweet_nuevo)
print("Tweet nuevo:", tweet_nuevo)
print("Predicción:", prediccion[0])

Tweet nuevo: ["I'm at the office working on a new project."]
Predicción: pos
